In [ ]:
import json
from langchain_openai import OpenAIEmbeddings
from functions import inference

In [ ]:
with open("problem_set_g7.json", "r") as f:
    data = json.load(f)

In [ ]:
n = len(data)
similar_score = [[0 for _ in range(n)] for _ in range(n)]
embedding_model = OpenAIEmbeddings(model='text-embedding-3-large')

In [ ]:
extract_prompt = r"""
    请从以下数学题目中提取关键信息。请返回以下格式的信息：

    1. 数学领域和类型（如代数、几何、微积分、统计等）
    2. 核心概念和原理（列出题目所需要的数学概念，如比例、相似、函数性质等）
    3. 解题方法和策略（列出用到特定的解题技巧，如配方法、换元法、归纳法等）
    4. 问题背景和情境（指出题目问题情境，如工程问题、经济问题、自然现象解释）
    5. 变量数量（题目所用到变量的数量，一定不要输出变量具体是什么）
    6. 题目所属年级（列出这道题最可能出现在哪一年级的试卷上）

    示例题目：
    求解方程 2x + 3 = 7

    示例输出：
    1. 数学领域和类型: 代数
    2. 核心概念和原理: 方程
    3. 解题方法和策略: 移项
    4. 问题背景和情境: 无
    5. 变量数量: 1
    6. 题目所属年级: 4

    现在请处理以下题目：
    {problem}
"""

In [ ]:
problems = []
for item in data:
    problems.append(item['problem'])
len(problems)

In [ ]:
import re
problems_emb = []
cache = {}
for problem in problems:
    attribs = []
    embedds = []
    temp = await inference({"problem": problem}, extract_prompt)
    pattern = r'\d+\.\s.*?:\s*(.*)'
    matches_p1 = re.findall(pattern, temp)
    for i, match in enumerate(matches_p1, 1):
        attribs.append(match)
    for att in attribs:
        if att in cache:
            embedds.append(cache.get(att))
        else:
            eb = embedding_model.embed_query(att)
            embedds.append(eb)
            cache[att] = eb
    problems_emb.append(embedds)

In [ ]:
len(problems_emb)

In [ ]:
with open("problems_emb.json", "w", encoding='utf-8') as f:
    json.dump(problems_emb, f, ensure_ascii=False, indent=4)

In [ ]:
problem_q = "估计与根号20最接近的一个整数是多少？"
q_emb = []
q_att = []
temp = await inference({"problem": problem_q}, extract_prompt)
pattern = r'\d+\.\s.*?:\s*(.*)'
matches_p1 = re.findall(pattern, temp)

for i, match in enumerate(matches_p1, 1):
    q_att.append(match)
    q_emb.append(embedding_model.embed_query(match))
q_emb

In [ ]:
import torch
q_emb = torch.tensor(q_emb)

In [ ]:
problems_emb = torch.tensor(problems_emb)
problems_emb.shape

In [ ]:
weights = [0.4, 0.2, 0.15, 0.05, 0.05, 0.15]
record = []
for i in range(problems_emb.shape[0]):
    saved_problem = problems_emb[i]
    similar_score = 0
    for j in range(q_emb.shape[0]):
        similar_score += torch.dot(q_emb[j], saved_problem[j]) * weights[j]
    record.append(similar_score)
record

In [ ]:
t = record.index(max(record))

In [ ]:
problems[t]

In [1]:
from functions import search_similar
await search_similar("估计与根号20最接近的一个整数是多少？")

executing
1. 数学领域和类型: 数论
2. 核心概念和原理: 平方根、整数估计
3. 解题方法和策略: 近似估计、平方数比较
4. 问题背景和情境: 无
5. 变量数量: 1
6. 题目所属年级: 6

(tensor(0.9501), 15)

In [3]:
import json
with open("problem_set_g7.json", 'r') as f:
    data = json.load(f)
data[15]

{'problem': '估计与根号十最接近的一个整数是多少？',
 'answer': '根号十是介于三和四之间的一个数，那这道题的意思是要你分析根号十是更靠近于三呢还是更靠近于四呢？所以本题的关键就是需要对根号十做更精确一些的估值。好我们从3.5开始，3.5的平方等于12.25比十大了，所以根号十是小于3.5，也即是最接近根号十的整数是三。'}